# Here is where anyone can input a decimated WAV file and receieve a TXT file that contains bounding box locations.

In [1]:
# Only run once
!pip install import_ipynb
!pip install opencv-python
!pip install librosa
!pip install tensorflow
!pip install tensorflow_probability
!pip install tensorflow_addons
!pip install scikit-maad

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [4]:
# Imports
import import_ipynb
import tensorflow
from tensorflow import keras
import boto3
import pandas as pd
import numpy as np
import os


s3 = boto3.resource('s3')

# S3 Bucket for Professor's Account is 'monitoring-whale-recordings'
# S3 Bucket for our free tier Account is 'monitoring-whale-records'
bucket_name = 'monitoring-whale-recordings'
bucket = s3.Bucket(bucket_name)

# Remove the previous _finalPreds.txt file if it was not erased yet
os.system(f'rm -rf *_finalPreds.txt')

0

In [5]:
# Welcome message
# User input (decimated WAV file)
wfile = input("Enter decimated WAV file (i.e., ###.#####_processed.wav): ")

Enter decimated WAV file (i.e., ###.#####_processed.wav): 671658014.180929123551_processed.wav


In [6]:
# Implement PCEN

# Need preprocessing functions
%run preprocessing.ipynb

for wf in WAV_FILES:
    if wfile == wf[0]:
        # Download WAV file from S3 Bucket
        s3.meta.client.download_file(bucket_name, wf[1], wf[0])

wfile_pcen, sr = process_wav(wfile, running=True)

In [7]:
# Load model and get initial boxes

# Need model functions
%run model.ipynb

vae = keras.models.load_model("test_vae_mod_pcen", custom_objects={"vae_loss_function": vae_loss_function})
non_normal_scores = error_dataset(vae, wfile_pcen, False, sr=sr)
bounding_boxes = run_model(non_normal_scores)
titles = ["Begin Time (s)", "End Time (s)", "Low Freq (Hz)", "High Freq (Hz)", "Species confidence"]
wfile_preds = wfile.split('_')[0] + "_predictions.txt"
write_array_to_file(bounding_boxes, titles, wfile_preds)

/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


36/36 [==============================] - 18s 399ms/step


In [8]:
# Finalize boxes from model

# Need postprocessing functions
%run postprocessing.ipynb

output = read_boxes(wfile_preds, True)

fOut = nms(
    filterBoxes(
        filterBoxes(
            filterBoxes(
                output,
                dim = 'top',
                upper = False,
                thresh = 2400
            ), 
            thresh = 0.3
        ),
        dim = 'dur',
        thresh = 0.25
        
    ), 
    0.5
)

In [9]:
# Convert output into TXT file

df = pd.DataFrame.from_dict(fOut, orient='index')
wfile_nums = wfile.split('_')[0]
np.savetxt(wfile_nums + "_finalPreds.txt", df.sort_index(), fmt='%s',
           delimiter='\t', header=', '.join(map(str,titles)))
print(f'{wfile_nums}_finalPreds.txt has been created.')

# Remove the _predictions.txt and _processed.wav (no longer needed)
os.system(f'rm -rf {wfile_nums}_predictions.txt {wfile_nums}_processed.wav')

671658014.180929123551_finalPreds.txt has been created.


0